# 概要
- trainとtestのbbをまとめる（indexずれを防ぐため）
- BB1のscaffoldの情報を使って、FOLDを設定する
- bb1~3と同様にbb1もindex化して列を足し、scaffold用の辞書も準備

In [1]:
import os
from pathlib import Path

def is_kaggle_kernel():
    return os.path.exists('/kaggle/working')

if is_kaggle_kernel():

    BASE_DIR = Path("/kaggle")
    DATA_DIR = BASE_DIR / "input"
    OUTPUT_DIR = BASE_DIR / "working"
    print('on kaggle notebook')

else:
    BASE_DIR = Path(os.getcwd()) / './../'
    DATA_DIR = BASE_DIR / "data"
    OUTPUT_DIR = BASE_DIR / "output/eda"
    
class paths:    
    DATA_DIR = DATA_DIR
    TRAIN_PATH = DATA_DIR / "train.parquet"
    TEST_PATH = DATA_DIR / "test.parquet"
    OUTPUT_DIR = OUTPUT_DIR
    SHRUNKEN_DATA_DIR = DATA_DIR / "shrunken-train-set"
    
    OUTPUT_DIR.mkdir(exist_ok=True, parents=True)

# 必要なデータ
- smiles to indexの辞書, index to smilesの辞書
    - bb1
    - bb1 scaffold
    - bb2, 3 scaffold
    - bb2, 3 scaffold
    - main moleculeの scaffoldの辞書

- shrunkenしたテストデータ
    - train
    - test（pretrain用）
- non shrunken test（submitt用）

In [2]:
# !pip install rdkit

In [3]:
import os
import gc
import math
import numpy as np
import pandas as pd
from glob import glob
# import duckdb
# import lightgbm as lgb
from tqdm import tqdm
from rdkit import Chem
from rdkit.Chem import Draw

import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from rdkit.Chem.Scaffolds import MurckoScaffold

import pickle

In [4]:
bb_cols = ['buildingblock1_smiles', 'buildingblock2_smiles','buildingblock3_smiles', 
        #    'molecule_smiles'
           ]
TARGETS = ['binds_BRD4', 'binds_HSA','binds_sEH']

df_train = pd.read_parquet(paths.DATA_DIR / 'shrunken-train-set/train.parquet', columns=bb_cols + TARGETS)
df_test = pd.read_parquet(paths.DATA_DIR / 'shrunken-train-set/test.parquet', columns=bb_cols)

# submit用のtestデータ
df_sub = pd.read_parquet(paths.DATA_DIR / 'test.parquet', columns=['id']+bb_cols)

In [5]:
# building block smiles
# NOTE: trainとtestのindexとsmilesは一致していないっぽい
with open(paths.SHRUNKEN_DATA_DIR / 'train_dicts/BBs_dict_reverse_1.p', 'rb') as file:
    train_dicts_bb1 = pickle.load(file)
with open(paths.SHRUNKEN_DATA_DIR / 'train_dicts/BBs_dict_reverse_2.p', 'rb') as file:
    train_dicts_bb2 = pickle.load(file)
with open(paths.SHRUNKEN_DATA_DIR / 'train_dicts/BBs_dict_reverse_3.p', 'rb') as file:
    train_dicts_bb3 = pickle.load(file)

with open(paths.SHRUNKEN_DATA_DIR / 'test_dicts/BBs_dict_reverse_1_test.p', 'rb') as file:
    test_dicts_bb1 = pickle.load(file)
with open(paths.SHRUNKEN_DATA_DIR / 'test_dicts/BBs_dict_reverse_2_test.p', 'rb') as file:
    test_dicts_bb2 = pickle.load(file)
with open(paths.SHRUNKEN_DATA_DIR / 'test_dicts/BBs_dict_reverse_3_test.p', 'rb') as file:
    test_dicts_bb3= pickle.load(file)

一旦smilesに直しておく

In [6]:
# 一旦smilesに直しておく
df_train['buildingblock1_smiles'] = df_train['buildingblock1_smiles'].map(train_dicts_bb1)
df_train['buildingblock2_smiles'] = df_train['buildingblock2_smiles'].map(train_dicts_bb2)
df_train['buildingblock3_smiles'] = df_train['buildingblock3_smiles'].map(train_dicts_bb3)
df_test['buildingblock1_smiles'] = df_test['buildingblock1_smiles'].map(test_dicts_bb1)
df_test['buildingblock2_smiles'] = df_test['buildingblock2_smiles'].map(test_dicts_bb2)
df_test['buildingblock3_smiles'] = df_test['buildingblock3_smiles'].map(test_dicts_bb3)

## smilet to idxの辞書を作成
trainとtestを合わせた辞書を作成する

In [7]:
bb1_smiles_train = [smiles for smiles in train_dicts_bb1.values()]
bb2_smiles_train = [smiles for smiles in train_dicts_bb2.values()]
bb3_smiles_train = [smiles for smiles in train_dicts_bb3.values()]
bb1_smiles_test = [smiles for smiles in test_dicts_bb1.values()]
bb2_smiles_test = [smiles for smiles in test_dicts_bb2.values()]
bb3_smiles_test = [smiles for smiles in test_dicts_bb3.values()]

bb1_smiles = sorted(list(set(bb1_smiles_train) | set(bb1_smiles_test)))
bb23_smiles = sorted(list(set(bb2_smiles_train) | set(bb2_smiles_test) | set(bb3_smiles_train) | set(bb3_smiles_test)))

print(len(bb1_smiles), len(bb23_smiles))

# smilesをindexに変換する辞書を作成
bb1_smiles2idx = {smiles: idx for idx, smiles in enumerate(bb1_smiles)}
bb23_smiles2idx = {smiles: idx for idx, smiles in enumerate(bb23_smiles)}


341 1769


## scaffold smilesを作成

In [8]:
# bb1のscaffoldを取得
bb1_scafold_dict = {}
for smiles in tqdm(bb1_smiles):
    mol = Chem.MolFromSmiles(smiles)
    scaffold = MurckoScaffold.GetScaffoldForMol(mol)
    scaffold_smiles = Chem.MolToSmiles(scaffold)
    
    if scaffold_smiles == '':
        scaffold_smiles = 'C'
    
    bb1_scafold_dict[smiles] = scaffold_smiles

bb23_scafold_dict = {}
for smiles in tqdm(bb23_smiles):
    mol = Chem.MolFromSmiles(smiles)
    scaffold = MurckoScaffold.GetScaffoldForMol(mol)
    scaffold_smiles = Chem.MolToSmiles(scaffold)
    
    if scaffold_smiles == '':
        scaffold_smiles = 'C'
    
    bb23_scafold_dict[smiles] = scaffold_smiles

# scaffoldのリスト
bb1_scaffold_smiles = list(set([smiles for smiles in bb1_scafold_dict.values()]))
bb23_scaffold_smiles = list(set([smiles for smiles in bb23_scafold_dict.values()]))

# scaffoldのsmilesをindexに変換する辞書を作成
bb1_scaffold_smiles2idx = {smiles: idx for idx, smiles in enumerate(bb1_scaffold_smiles)}
bb23_scaffold_smiles2idx = {smiles: idx for idx, smiles in enumerate(bb23_scaffold_smiles)}

100%|██████████| 1769/1769 [00:00<00:00, 18599.90it/s]


In [9]:
df_train["buildingblock1_smiles_scaffold"] = df_train["buildingblock1_smiles"].map(bb1_scafold_dict)
df_train["buildingblock2_smiles_scaffold"] = df_train["buildingblock2_smiles"].map(bb23_scafold_dict)
df_train["buildingblock3_smiles_scaffold"] = df_train["buildingblock3_smiles"].map(bb23_scafold_dict)

df_test["buildingblock1_smiles_scaffold"] = df_test["buildingblock1_smiles"].map(bb1_scafold_dict)
df_test["buildingblock2_smiles_scaffold"] = df_test["buildingblock2_smiles"].map(bb23_scafold_dict)
df_test["buildingblock3_smiles_scaffold"] = df_test["buildingblock3_smiles"].map(bb23_scafold_dict)

df_sub["buildingblock1_smiles_scaffold"] = df_sub["buildingblock1_smiles"].map(bb1_scafold_dict)
df_sub["buildingblock2_smiles_scaffold"] = df_sub["buildingblock2_smiles"].map(bb23_scafold_dict)
df_sub["buildingblock3_smiles_scaffold"] = df_sub["buildingblock3_smiles"].map(bb23_scafold_dict)

## 分子をscaffoldごとにクラスタリングする

In [10]:
# bb1のscaffoldを取得
scaffold_list = []
for smiles in tqdm(bb1_smiles_train):
    mol = Chem.MolFromSmiles(smiles)
    scaffold = MurckoScaffold.GetScaffoldForMol(mol)
    scaffold_smiles = Chem.MolToSmiles(scaffold)
    
    scaffold_list.append(scaffold_smiles)
    
scaffold_list = sorted(list(set(scaffold_list)))

  0%|          | 0/271 [00:00<?, ?it/s]

100%|██████████| 271/271 [00:00<00:00, 5798.59it/s]


In [11]:
# タニモト係数に応じてグループ分け
from rdkit.Chem import AllChem
from rdkit import DataStructs

scaffold_mols = [Chem.MolFromSmiles(s) for s in scaffold_list]
morgan_fp = [AllChem.GetMorganFingerprintAsBitVect(x, 2, 2048) for x in scaffold_mols]

dis_matrix = [DataStructs.BulkTanimotoSimilarity(morgan_fp[i], morgan_fp, returnDistance=True) for i in range(len(morgan_fp))]
### numpy.ndarrayへの変換
dis_array = np.array(dis_matrix)
dis_array.shape ### (5000, 5000)

(63, 63)

In [12]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=5, random_state=5)
kmeans.fit(dis_array)

print('各クラスタのscaffold数')
print(pd.value_counts(kmeans.labels_))

fold_scaffold_dict = {i: [] for i in range(5)}
for n,j in enumerate(kmeans.labels_):
    fold_scaffold_dict[j].append(scaffold_list[n])
    
# fold1のscaffoldの4番目だけをfold0にして、それ以外ををfold0に押し込む（圧倒的に数が多いので）
num = 4
fold_scaffold_dict[0].extend(fold_scaffold_dict[1][:num] + fold_scaffold_dict[1][num+1:])
fold_scaffold_dict[1] = [fold_scaffold_dict[1][num]]

print('各クラスタのscaffold数(修正後)')
for fold, s_list in fold_scaffold_dict.items():
    print(fold, len(s_list))

fold_scaffold_dict_reverse = {}
# key, valの入れ替え
for fold, s_list in fold_scaffold_dict.items():
    for s in s_list:
        fold_scaffold_dict_reverse[s] = fold

各クラスタのscaffold数
1    19
2    16
4    15
3    12
0     1
Name: count, dtype: int64
各クラスタのscaffold数(修正後)
0 19
1 1
2 16
3 12
4 15


/usr/local/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/tmp/ipykernel_66572/3102218544.py:6: FutureWarning: pandas.value_counts is deprecated and will be removed in a future version. Use pd.Series(obj).value_counts() instead.
  print(pd.value_counts(kmeans.labels_))


In [13]:
# foldを分ける
df_train["fold"] = df_train["buildingblock1_smiles_scaffold"].map(fold_scaffold_dict_reverse)
display(df_train['fold'].value_counts())

# >>> fold2のbinds_sEHだけ以上に多い
for fold in range(5):
    print(f'fold{fold}')
    print(df_train[df_train['fold'] == fold][TARGETS].mean())
    print('---')

fold
1    31600668
4    27966077
0    19967499
2    10163121
3     8718245
Name: count, dtype: int64

fold0
binds_BRD4    0.006064
binds_HSA     0.004038
binds_sEH     0.002274
dtype: float64
---
fold1
binds_BRD4    0.003272
binds_HSA     0.004994
binds_sEH     0.002603
dtype: float64
---
fold2
binds_BRD4    0.004564
binds_HSA     0.002907
binds_sEH     0.044195
dtype: float64
---
fold3
binds_BRD4    0.002707
binds_HSA     0.003055
binds_sEH     0.001804
dtype: float64
---
fold4
binds_BRD4    0.005811
binds_HSA     0.004069
binds_sEH     0.004720
dtype: float64
---


## 全てIDXに変換して、変換用の辞書を保存する

In [14]:
col_order = [
    'buildingblock1_smiles', 'buildingblock2_smiles','buildingblock3_smiles', 
       'buildingblock1_smiles_scaffold', 'buildingblock2_smiles_scaffold','buildingblock3_smiles_scaffold',
       'fold',
       'binds_BRD4', 'binds_HSA', 'binds_sEH']
df_train = df_train[col_order]
df_test = df_test[col_order[:-4]]

In [15]:
# 全てindexに変換する
def smiles2idx(df):
    df = df.copy()
    df['buildingblock1_smiles'] = df['buildingblock1_smiles'].map(bb1_smiles2idx)
    df['buildingblock2_smiles'] = df['buildingblock2_smiles'].map(bb23_smiles2idx)
    df['buildingblock3_smiles'] = df['buildingblock3_smiles'].map(bb23_smiles2idx)
    df['buildingblock1_smiles_scaffold'] = df['buildingblock1_smiles_scaffold'].map(bb1_scaffold_smiles2idx)
    df['buildingblock2_smiles_scaffold'] = df['buildingblock2_smiles_scaffold'].map(bb23_scaffold_smiles2idx)
    df['buildingblock3_smiles_scaffold'] = df['buildingblock3_smiles_scaffold'].map(bb23_scaffold_smiles2idx)
    return df

df_train = smiles2idx(df_train)
df_test = smiles2idx(df_test)
df_sub = smiles2idx(df_sub)


In [16]:
df_sub.head()

,id,buildingblock1_smiles,buildingblock2_smiles,buildingblock3_smiles,buildingblock1_smiles_scaffold,buildingblock2_smiles_scaffold,buildingblock3_smiles_scaffold
0,295246830,0,20,20,73,105,105
1,295246831,0,20,20,73,105,105
2,295246832,0,20,20,73,105,105
3,295246833,0,20,119,73,105,55
4,295246834,0,20,119,73,105,55


In [17]:
## 保存
SAVE_DIR = paths.DATA_DIR / 'shrunken-data'
SAVE_DIR.mkdir(exist_ok=True, parents=True)

df_train.to_parquet(SAVE_DIR / 'train.parquet')
df_test.to_parquet(SAVE_DIR / 'test.parquet')
df_sub.to_parquet(SAVE_DIR / 'sub.parquet')

with open(SAVE_DIR / 'bb1_smiles2idx.pickle', mode='wb') as f:
  pickle.dump(bb1_smiles2idx, f)
with open(SAVE_DIR / 'bb23_smiles2idx.pickle', mode='wb') as f:
    pickle.dump(bb23_smiles2idx, f)
with open(SAVE_DIR / 'bb1_scaffold_smiles2idx.pickle', mode='wb') as f:
    pickle.dump(bb1_scaffold_smiles2idx, f)
with open(SAVE_DIR / 'bb23_scaffold_smiles2idx.pickle', mode='wb') as f:
    pickle.dump(bb23_scaffold_smiles2idx, f)